In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# the folder where you want to save all matrix
SAVE_DIR = "Social/FC1/"

# name of your task
HCP_DIR = "hcp_task/"
task = "SOCIAL"
subjects = np.loadtxt(os.path.join(HCP_DIR, 'subjects_list.txt'), dtype='str')

In [3]:
# do not have to change this cell

RUNS = ['LR','RL']

# fixed
N_SUBJECTS = 100
N_PARCELS = 360
TR = 0.72
N_RUNS = 2

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['mental','rnd']}
}

In [4]:
# do not have to change this cell
# functions copied from nma

def load_single_timeseries(subject, experiment, run, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (str):      subject ID to load
    experiment (str):   Name of experiment
    run (int):          (0 or 1)
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run  = RUNS[run]
  bold_path = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
  bold_file = "data.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts


def load_evs(subject, experiment, run):
  """Load EVs (explanatory variables) data for one task experiment.

  Args:
    subject (str): subject ID to load
    experiment (str) : Name of experiment
    run (int): 0 or 1

  Returns
    evs (list of lists): A list of frames associated with each condition

  """
  frames_list = []
  task_key = f'tfMRI_{experiment}_{RUNS[run]}'
  for cond in EXPERIMENTS[experiment]['cond']:
    ev_file  = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)
    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)
    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
    frames_list.append(frames)

  return frames_list

In [14]:
for i_sub in range(len(subjects)):
    sub = subjects[i_sub]

    for i_run in range(2):

        run = RUNS[i_run]
        dir_data = os.path.join(HCP_DIR, 'subjects', sub, task, 'tfMRI_'+task+'_'+run, 'data.npy')
        data = np.load(dir_data)
        evs = load_evs(subject=sub, experiment=task, run=i_run)
        # do not have to change the code above

        ts_cond1_run = np.concatenate([data[:, evs[0][i]] for i in range(len(evs[0]))], axis=1)
        ts_cond2_run = np.concatenate([data[:, evs[1][i]] for i in range(len(evs[1]))], axis=1)
        # if you have condition 5, notice the part range(len(evs[4])), where 4 = 5-1
        # ts_cond5_run = np.concatenate([data[:, evs[4][i]] for i in range(len(evs[4]))], axis=1)   

        if i_run == 0:
            ts_cond1 = ts_cond1_run
            ts_cond2 = ts_cond2_run
            #ts_cond5 = ts_cond5_run
        else:
            ts_cond1 = np.concatenate([ts_cond1, ts_cond1_run], axis=1)
            ts_cond2 = np.concatenate([ts_cond2, ts_cond2_run], axis=1)
            #ts_cond5 = np.concatenate([ts_cond5, ts_cond5_run], axis=1)

    fc_con1 = np.corrcoef(ts_cond1)
    fc_con2 = np.corrcoef(ts_cond2)
    #fc_con5 = np.corrcoef(ts_cond5)


    save_name_con1 = "sub_"+sub+"_"+task+"_"+EXPERIMENTS[task]['cond'][0]+".npy"
    np.save(SAVE_DIR+save_name_con1, fc_con1)
    print(save_name_con1)
    
    save_name_con2 = "sub_"+sub+"_"+task+"_"+EXPERIMENTS[task]['cond'][1]+".npy"
    np.save(SAVE_DIR+save_name_con2, fc_con2)
    print(save_name_con2)
    
    # if you have condition 5, notice the part EXPERIMENTS[task]['cond'][4], where 4 = 5-1
    # save_name_con5 = "sub_"+sub+"_"+task+"_"+EXPERIMENTS[task]['cond'][4]+".npy"
    # np.save(SAVE_DIR+save_name_con5, fc_con5)
    # print(save_name_con5)

print("all finished")

sub_100307_SOCIAL_mental.npy
sub_100307_SOCIAL_rnd.npy
sub_100408_SOCIAL_mental.npy
sub_100408_SOCIAL_rnd.npy
sub_101915_SOCIAL_mental.npy
sub_101915_SOCIAL_rnd.npy
sub_102816_SOCIAL_mental.npy
sub_102816_SOCIAL_rnd.npy
sub_103414_SOCIAL_mental.npy
sub_103414_SOCIAL_rnd.npy
sub_103515_SOCIAL_mental.npy
sub_103515_SOCIAL_rnd.npy
sub_103818_SOCIAL_mental.npy
sub_103818_SOCIAL_rnd.npy
sub_105115_SOCIAL_mental.npy
sub_105115_SOCIAL_rnd.npy
sub_105216_SOCIAL_mental.npy
sub_105216_SOCIAL_rnd.npy
sub_106016_SOCIAL_mental.npy
sub_106016_SOCIAL_rnd.npy
sub_106319_SOCIAL_mental.npy
sub_106319_SOCIAL_rnd.npy
sub_110411_SOCIAL_mental.npy
sub_110411_SOCIAL_rnd.npy
sub_111009_SOCIAL_mental.npy
sub_111009_SOCIAL_rnd.npy
sub_111312_SOCIAL_mental.npy
sub_111312_SOCIAL_rnd.npy
sub_111514_SOCIAL_mental.npy
sub_111514_SOCIAL_rnd.npy
sub_111716_SOCIAL_mental.npy
sub_111716_SOCIAL_rnd.npy
sub_113215_SOCIAL_mental.npy
sub_113215_SOCIAL_rnd.npy
sub_113619_SOCIAL_mental.npy
sub_113619_SOCIAL_rnd.npy
sub_114924